**This is the final notebook for Milestone 3, where all the data analysis (besides the pre processing done in Milestone 2) is done.**

In [ ]:
#!pip install bokeh

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import timeit
import bz2
import datetime
import os
from src.prep_utilities import * 
from src.prep_pipeline import *

%matplotlib inline
%load_ext autoreload
%autoreload 2
!python ./src/load_models_data.py
data_folder = './data/'

# General Gender Bias Analyses

In this section, we plan to analyse:
 - Evolution of percentage of speakers by gender, over time
 - Evolution of percentage of quotations by gender, over time
 - Most quoted speakers by gender, over time

We'll do a month-by-month analysis.

We will save the total number of quotations by gender (`['numOccurrences'].sum()`), the number of speakers of each gender (using `count()`) and the percentages to a dictionary of lists. Each key will be the gender and each list will have the data corresponding to each month (in 

In [23]:
# dictionary to keep counts for different genders, the arrays start at 2015-01 and end at 2020-04, in increments of 1 month
gender_count = {"male": [], "female": [], "transgender female": [], "non-binary": [], "genderfluid": [], "transgender male": []} 

# dictionary to keep percentages
gender_percent = {"male": [], "female": [], "transgender female": [], "non-binary": [], "genderfluid": [], "transgender male": []} 

The pre-processed data will be loaded each year, by chunks. In the analyses, we will cycle through each year, and through each chunk, and save the data.

In [25]:
years = [2019] ###CHANGE THIS  FOR ALL YEARS

In [27]:
start = timeit.default_timer()
# Loop through years
for year in years:
    
    # data location and chunk size
    data_file = 'quotes-'+ str(year)+'-prep.json.bz2'
    data_path = data_folder + data_file
    chunk_size = 1e4

    # Load by chunks
    f = bz2.open(data_path, "rb")
    data=pd.read_json(f, lines=True, chunksize=chunk_size)
    
    # Loop through chunks
    for i_chunk, chunk in enumerate(data):
        
        ## Test the speed of this
        print(f"Male quotes in chunk {i_chunk} = {chunk[chunk['gender'] == 'male']['numOccurrences'].sum()}")

stop = timeit.default_timer()
print(f"Time = {stop-start:.2f}s")

Male quotes in chunk 0 = 26715
Male quotes in chunk 1 = 26788
Male quotes in chunk 2 = 26103
Male quotes in chunk 3 = 26613
Male quotes in chunk 4 = 26337
Male quotes in chunk 5 = 27728
Male quotes in chunk 6 = 26061
Male quotes in chunk 7 = 31584
Male quotes in chunk 8 = 25209
Male quotes in chunk 9 = 28079
Male quotes in chunk 10 = 29463
Male quotes in chunk 11 = 26563
Male quotes in chunk 12 = 26001
Male quotes in chunk 13 = 25478
Male quotes in chunk 14 = 26326
Male quotes in chunk 15 = 26723
Male quotes in chunk 16 = 29391
Male quotes in chunk 17 = 27995
Male quotes in chunk 18 = 27617
Male quotes in chunk 19 = 25988
Male quotes in chunk 20 = 25685
Male quotes in chunk 21 = 28189
Male quotes in chunk 22 = 24868
Male quotes in chunk 23 = 27208
Male quotes in chunk 24 = 27822
Male quotes in chunk 25 = 27258
Male quotes in chunk 26 = 27533
Male quotes in chunk 27 = 25350
Male quotes in chunk 28 = 25860
Male quotes in chunk 29 = 27117
Male quotes in chunk 30 = 27075
Male quotes in chu